In [2]:
import numpy as np
import pandas as pd
import networkx as nx

I cant run the complete 65 million interactions dataset on my system. So I am running it on a very small dataset. So the accuracy i got is very bad. To get better results the graph should have sufficient number of connections.

In [5]:
interactions = pd.read_csv("data_train_subset.csv")

In [6]:
interactions.head()

,users,jobs,event,timestamps
0,2531442,93409,click,1582436367
1,2038815,168399,click,1582597367
2,2678041,122067,click,1582407982
3,1073494,9017,click,1581885544
4,2727150,92678,bookmark,1582145466


In [7]:
edge_list = list(zip(interactions['users'], interactions['jobs']))
edge_list

[(2531442, 93409),
 (2038815, 168399),
 (2678041, 122067),
 (1073494, 9017),
 (2727150, 92678),
 (2425745, 58619),
 (2966695, 164856),
 (304367, 50672),
 (2575625, 41641),
 (1277620, 3689),
 (2461158, 136590),
 (1313897, 17578),
 (391052, 139358),
 (459183, 79185),
 (845622, 50997),
 (912323, 16460),
 (1864680, 126505),
 (1267122, 159045),
 (3071466, 108470),
 (3138116, 130872),
 (1335578, 183372),
 (1867604, 177730),
 (2489078, 73430),
 (31878, 175708),
 (2315396, 51028),
 (177156, 179430),
 (1860554, 662),
 (2397802, 59743),
 (494125, 60651),
 (2903785, 40892),
 (3088779, 163131),
 (2295484, 126232),
 (1907378, 181147),
 (1878731, 172768),
 (948975, 112360),
 (1924305, 86784),
 (238990, 55875),
 (1662565, 134876),
 (2848128, 17686),
 (989482, 1067),
 (2006696, 42307),
 (304215, 36107),
 (2977821, 183627),
 (1830089, 128015),
 (2887232, 79955),
 (2813303, 100532),
 (962545, 105732),
 (2246382, 34989),
 (862017, 67506),
 (1291103, 140670),
 (2726758, 49212),
 (1308105, 160831),
 (12159

In [8]:
unique_edges = list(set(edge_list))
len(unique_edges)

1000

bipartite graph with all the users on one bipartite and all the jobs on other bipartite

In [9]:
G = nx.Graph()
jobs = pd.Series(interactions.jobs).unique()
users = pd.Series(interactions.users).unique()
G.add_nodes_from(users, bipartite=0, label='users')
G.add_nodes_from(jobs, bipartite=1, label='jobs')
G.add_edges_from(unique_edges)
# G.nodes(data=True)

In [52]:
partition_0_nodes= users.tolist()
partition_0_node_count = len(partition_0_nodes)

partition_1_nodes= jobs.tolist()
partition_1_node_count = len(partition_1_nodes)

# Print the node counts
print("Partition 0 node count:", partition_0_node_count)
print("Partition 1 node count:", partition_1_node_count)

Partition 0 node count: 999
Partition 1 node count: 994


indexing users and jobs for the creation of r_score matrix

In [61]:
users_index = list(enumerate(partition_0_nodes, start=0))
jobs_index=list(enumerate(partition_1_nodes, start=0))

In [74]:
users_dict= dict(users_index)
users_dict= {value: key for key, value in users_dict.items()}
serial_to_job_dict= dict(jobs_index)
jobs_dict= {value: key for key, value in serial_to_job_dict.items()}

In [64]:
r_score = np.zeros((partition_0_node_count, partition_1_node_count))

In [65]:
def fill_matrix(node1,u_dash,i_dash,node2,alpha,beta):
    p_u_idash= 1/ abs(len(list(G.neighbors(node1))))**alpha
    p_idash_udash= 1/ abs(len(list(G.neighbors(i_dash))))**alpha
    p_udash_i= 1/ ( (abs(len(list(G.neighbors(u_dash))))**alpha) * (abs(len(list(G.neighbors(node2))))**beta) )
    return (p_u_idash*p_idash_udash*p_udash_i)

In [66]:
def sim_score(node1,node2,alpha,beta):
    r_score=0;
    for i_dash in list(G.neighbors(node1)):
        for u_dash in list(G.neighbors(i_dash)):
             if u_dash!=node1 and G.has_edge(u_dash, node2):
                 r_score+=fill_matrix(node1,u_dash,i_dash,node2,alpha,beta)
    return r_score
# print(sim_score(50,1006))

In [80]:
def cal_sim_score_matrix(alpha,beta):
    for node1,x in users_dict.items():
        for node2,y in jobs_dict.items():
            r_score[x][y]+= sim_score(node1,node2,alpha,beta)
    print(r_score)

the best values of the parameters alpha and beta can be found by cross validation. The values i took as alpha and beta are from the research paper.

In [81]:
alpha=0.61447198
beta= 0.1443548
cal_sim_score_matrix(alpha,beta)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.18195345 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [82]:
def new_user_recommendations(n):
    # Calculate item popularity (counting the number of interactions)
    item_popularity = interactions['jobs'].value_counts().reset_index()
    item_popularity.columns = ['item', 'popularity']
    sorted_items = item_popularity.sort_values('popularity', ascending=False)
    top_n_recommendations = sorted_items['item'].head(n).tolist()
    return top_n_recommendations

In [83]:
def recommendations(user_idx,n):
    if not (user_idx in users):
        key_list=list(new_user_recommendations(n))
        return key_list
    user_scores = r_score[users_dict.get(user_idx)]
    sorted_scores_indices = np.argsort(-user_scores)

    top_n_recommendation_list = sorted_scores_indices[:n]
    top_n_recommendations = [serial_to_job_dict.get(x) for x in top_n_recommendation_list]
    return top_n_recommendations

In [84]:
recommendations(2966695,10)

[93409, 172810, 100184, 75256, 128498, 139149, 20358, 57789, 51660, 67090]

In [85]:
new_user_recommendations(10)

[168399, 115366, 146291, 54370, 50277, 33678, 50672, 139358, 17578, 136590]

# test

In [86]:
test_data = pd.read_csv("data_test_subset.csv")
test_data.head()

,users,jobs,event,timestamps
0,2563823,98021,click,1581994931
1,2448122,122656,click,1581526970
2,310204,107928,click,1581579765
3,1118737,120433,click,1582587666
4,427433,12303,click,1581531903


In [97]:
total_users = test_data['users'].unique()
total_jobs = test_data['jobs'].unique()

precision_sum = 0
recall_sum = 0
print(len(total_users))
print(len(total_jobs))

80
80


In [98]:
new_user=0
new_job=0
for user in total_users:
    if user not in users:
        new_user=new_user+1
for job in total_jobs:
    if job not in jobs:
        new_job=new_job+1

print(new_user)
print(new_job)

80
75


In [100]:
for user_id in test_data['users'].unique():
    true_positives = 0
    recommended_jobs = recommendations(user_id, 50)
    user_items = test_data.loc[test_data['users'] == user_id, 'jobs']
    
    for item in recommended_jobs:
        if item in user_items:
            true_positives += 1
    
    precision = (true_positives / len(recommended_jobs)) if len(recommended_jobs) > 0 else 0
    recall = true_positives / len(user_items)
    
    precision_sum += precision
    recall_sum += recall
    
# Calculate average precision and recall
average_precision = precision_sum / len(total_users)
average_recall = recall_sum / len(total_users)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)

Average Precision: 0.0
Average Recall: 0.0
